In [1]:
import os
import pandas as pd
import numpy as np
from datetime import timedelta

In [2]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [3]:
# Decide backtesting period and montly rebalancing date
windowsize_for_calling = 120 
windowsize = 1 

startdate = pd.to_datetime('2001-01-01')

startdate_for_calling = startdate - timedelta(days=(windowsize_for_calling+1)) 

enddate = pd.to_datetime("2021-08-31")

#------------------------------------------------------------------------------

checklist = []  # Store rebalance date (day one of each month)

temp = startdate

while(1): # monthly rebalancing
    checklist.append(pd.to_datetime(temp))
    
    if temp.month ==12:
        temp = temp.replace(year = temp.year + 1,month = 1)
    else:
        temp = temp.replace(month = temp.month + 1)
        
    if temp > enddate:
        break

In [4]:
SP500_FULL = pd.read_csv('SP500_FULL.csv',index_col='Unnamed: 0')
SP500_FULL

,GR.N^G12,BAX.N,PWJ.N^K00,TRB.N^L07,MII.N^F99,VRTX.OQ,LYV.N,KR.N,UNP.N,MRO.N,...,SO.N,TGT.N,FMC.N,DELL.OQ^J13,BMET.OQ^J07,HSP.N^I15,CSR.N^F00,IDXX.OQ,WB.N^A09,SEE.N
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
1995-01-03,21.8750,7.639566,NaN,13.59375,NaN,NaN,NaN,3.046875,11.71875,5.046203,...,19.875,5.833333,6.765336,NaN,5.944439,NaN,22.750,NaN,20.8750,NaN
1995-01-04,22.1875,7.741427,NaN,13.62500,NaN,NaN,NaN,3.062500,11.71875,5.046203,...,20.125,5.822917,6.765336,NaN,6.277772,NaN,22.875,NaN,21.0625,NaN
1995-01-05,22.1250,7.741427,NaN,13.62500,NaN,NaN,NaN,3.109375,11.96875,5.084144,...,20.000,5.916667,6.779916,NaN,6.555549,NaN,22.875,NaN,21.0000,NaN
1995-01-06,21.9375,7.775380,NaN,13.53125,NaN,NaN,NaN,3.109375,12.09375,5.046203,...,20.125,5.875000,6.779916,NaN,6.499994,NaN,22.750,NaN,21.3125,NaN
1995-01-09,22.0000,7.673519,NaN,13.40625,NaN,NaN,NaN,3.125000,11.87500,5.008262,...,19.875,5.708333,6.809077,NaN,6.444438,NaN,22.875,NaN,21.3750,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-25,NaN,73.800000,NaN,NaN,NaN,201.33,85.89,45.800000,221.29000,11.800000,...,65.870,251.150000,92.900000,NaN,NaN,NaN,NaN,676.42,NaN,61.17
2021-08-26,NaN,73.770000,NaN,NaN,NaN,198.25,84.75,45.170000,220.98000,11.520000,...,66.140,248.960000,93.600000,NaN,NaN,NaN,NaN,673.36,NaN,60.67
2021-08-27,NaN,73.970000,NaN,NaN,NaN,199.92,86.90,45.490000,221.43000,12.010000,...,65.990,249.180000,93.070000,NaN,NaN,NaN,NaN,668.27,NaN,61.07


In [5]:
SP500_DELIST = pd.read_csv('S&P_delist.csv',encoding='cp949')
SP500_DELIST

,Ticker,Reason
0,UA/UAA,Market capitalization change.[5]
1,IPGP,Market capitalization change.[5]
2,CERN,S&P 500 constituent Oracle Corp. acquired Cern...
3,PBCT,S&P 500 constituent M&T Bank Corp. acquired Pe...
4,INFO,S&P 500 constituent S&P Global Inc. acquired I...
...,...,...
285,RAD,Market Cap change.[229]
286,TMC,Tribune Co. acquired Times Mirror.[230]
287,SMS,Siemens AG acquired Shared Medical Systems.[231]
288,LDW,Market capitalization change.[232]


In [6]:
positive = []
for row in SP500_DELIST.index:
    line =  SP500_DELIST.loc[row,'Reason']
    for keyword in ['acquired','spun off','Acquired','Spun off','Private','private','is being bought by','Company split','spin-off merged',
                   'split','acquires','merger','spins off','taken over','merged','separated','replaced']:
        if keyword in line:
            positive.append(SP500_DELIST.loc[row,'Ticker'])

In [7]:
path = 'subtraction/weight/'
file_list = os.listdir(path)
weight_list = [file for file in file_list if file.endswith('weight.csv')]
weight_list.sort(key=lambda x: int(x.split('_')[1]))
weight_list

['GM_5_weight.csv',
 'GM_10_weight.csv',
 'GM_20_weight.csv',
 'GM_30_weight.csv',
 'GM_50_weight.csv',
 'GM_100_weight.csv',
 'GM_200_weight.csv',
 'GM_300_weight.csv',
 'GM_500_weight.csv']

Portfolio Backtesting

In [8]:
for file in weight_list:
    if 'price' in locals():
        del price
    weight_data = pd.read_csv(path+file,index_col='Unnamed: 0')
    shift =  np.log(SP500_FULL/SP500_FULL.shift(periods=1))

    portfolio_value = 100
    fund_price = {}
    set_of_delisted = set()
    
    for rebalancedate_idx in range(len(checklist))[:]:
        #print('                    ')
        print('-------' + str(checklist[rebalancedate_idx])[:10] + '-------')
        
        tmp_idx = str(checklist[rebalancedate_idx])[:10]
        portfolio_weight = weight_data.loc[tmp_idx].dropna()
        investment_list = portfolio_weight.index
        
        if(rebalancedate_idx < len(checklist) - 1) : # 마지막 리밸런싱 시점이 아닌 경우
            sliced_data = shift[str(checklist[rebalancedate_idx])[:10]:str(checklist[rebalancedate_idx + 1])[:10]].copy() # 현 리밸런싱 시점에서 다음 리밸런싱 시점 사이의 종가 데이터를 가져온다. (다음 리밸런싱 시점까지 포함됨)

            if str(checklist[rebalancedate_idx + 1])[:10] in sliced_data.index: 
                sliced_data = sliced_data[:-1].dropna(axis = 1,how='all') # sliced_data에 다음 리밸런싱 시점이 포함이 되므로 해당 일 제외함.
            else:
                sliced_data = sliced_data[:].dropna(axis = 1,how='all') # sliced_data에 다음 리밸런싱 시점이 포함이 안되는 경우가 존재함 (ex: 3월 1일). 이 경우, [:-1]로 슬라이싱하게 되면 하루가 덜 투자되는 상황이 발생함. 

        else: # 마지막 리밸런싱 시점인 경우, 현 리밸런싱 시점에서 끝까지의 종가 데이터를 가져옴.
            sliced_data = shift[str(checklist[rebalancedate_idx])[:10]:str(enddate)].copy()
            sliced_data = sliced_data[:].dropna(axis = 1,how='all')
        
        sliced_data.index = pd.to_datetime(sliced_data.index)
        rebalancedate_in_workday = sliced_data.index[0]
        print("-"*30)

        log_ret = sliced_data.copy()
        print('                    ') 

        fund_price[log_ret.index[0]] = portfolio_value
        print(str(log_ret.index[0])[:10] + '--------' + str(portfolio_value))
        
        try:
            check_for_error = sliced_data.loc[rebalancedate_in_workday,investment_list]
        except: # 이 경우는 가중치는 계산이 되었으나, 실제 매수시점에 해당 종목이 없는 경우를 처리함. 해당 종목은 매수 종목 리스트에서 아예 제외되며, 그 투자비중은 cash_remained에 포함됨.
            for asset in portfolio_weight.index:
                if asset not in sliced_data.columns:
                    portfolio_weight = portfolio_weight.drop(asset)
        else:
            pass
        reserve_weight = 1- (portfolio_weight).sum() #투자 안된 남은 금액
        
        cash_reserve = portfolio_value*reserve_weight
        total_investment_of_each= portfolio_weight * portfolio_value
        
        for idx in range(0,log_ret.shape[0])[:]:
            fluctuation = np.exp(log_ret.iloc[idx])
            if fluctuation.isnull().any():
                delisted_asset = fluctuation[fluctuation.isnull()].index
                fluctuation = fluctuation.fillna(1)
                for asset in investment_list:
                    if asset in delisted_asset:
                        TICKER = asset.split('.')[0]
                        if TICKER in positive:
                            total_investment_of_each[asset] = total_investment_of_each[asset]
                        else:
                            total_investment_of_each[asset] = total_investment_of_each[asset] * 0.7
                            
                        investment_list = investment_list.drop(asset) # -30%는 한번만!
                        print(asset)
                    else:
                        pass
                total_investment_of_each = fluctuation * total_investment_of_each
                price = total_investment_of_each.sum() + cash_reserve
                
            else:
                total_investment_of_each = total_investment_of_each * fluctuation
                price = total_investment_of_each.sum() + cash_reserve
                
            fund_price[log_ret.index[idx]] = price
            portfolio_value = price

    fund_price = pd.DataFrame.from_dict(fund_price, orient='index',columns=['Price'])

    # Ensure the directory exists
    os.makedirs('subtraction/price', exist_ok=True)

    fund_price.to_csv('subtraction/price/'+file.replace('_weight.csv','')+'_price.csv')
    fund_price.index = pd.to_datetime(fund_price.index)

-------2001-01-01-------
------------------------------
                    
2001-01-02--------100
-------2001-02-01-------
------------------------------
                    
2001-02-01--------99.97130991401514
-------2001-03-01-------
------------------------------
                    
2001-03-01--------94.74634649493021
-------2001-04-01-------
------------------------------
                    
2001-04-02--------91.5267329537017
-------2001-05-01-------
------------------------------
                    
2001-05-01--------95.16835471514983
-------2001-06-01-------
------------------------------
                    
2001-06-01--------99.86176096898505
-------2001-07-01-------
------------------------------
                    
2001-07-02--------91.52312159316847
-------2001-08-01-------
------------------------------
                    
2001-08-01--------90.02825978915925
AGC.N^H01
-------2001-09-01-------
------------------------------
                    
2001-09-04--------72.396

In [9]:
path = 'subtraction/price/'
file_list = os.listdir(path)
price_list = [file for file in file_list]
price_list.sort(key=lambda x: int(x.split('_')[-2]))
price_list

['GM_5_price.csv',
 'GM_10_price.csv',
 'GM_20_price.csv',
 'GM_30_price.csv',
 'GM_50_price.csv',
 'GM_100_price.csv',
 'GM_200_price.csv',
 'GM_300_price.csv',
 'GM_500_price.csv']

In [10]:
def downside_risk(returns, risk_free=0):
    adj_returns = returns - risk_free
    sqr_downside = np.square(np.clip(adj_returns, np.NINF, 0))
    return np.sqrt(np.nanmean(sqr_downside) * 252)

def MDD(data):
    data = (data+1).cumprod()
    roll_max = data.cummax()
    daily_drawdown = data/roll_max -1.0
    MDD = daily_drawdown.cummin()
    return (min(MDD.values)) 

In [11]:
summary = pd.DataFrame(columns = price_list, index = ['Annualized compound return','Annualized Standard Deviation',\
                                                    'Annualized Downside Deviation','Maximum Drawdown','Sharpe Ratio','Sortino Ratio','Calmar Ratio'])
for fund_price in price_list:
    data = pd.read_csv(path+fund_price,index_col='Unnamed: 0')
    data.index = pd.to_datetime(data.index)
    
    log_ret = np.log(data/data.shift(periods=1))
    log_ret = log_ret.dropna()
    log_ret.index = data.index[1:]

    #Annualized coupound return
    cumsum = (data.values[-1] /data.values[0] -1)
    acr = cumsum/(len(log_ret.values) / 252)
    summary[fund_price][0] = round(acr[0], 4)

    #Annualized Standard Deviation
    stdev = np.std(log_ret.values)*np.sqrt(252)
    summary[fund_price][1] = round(stdev, 4)

    #Annualized Downside Deviation
    ADD = downside_risk(log_ret.values)
    summary[fund_price][2] = round(ADD, 4)

    #MDD
    mdd = MDD(log_ret)
    summary[fund_price][3] = round(mdd[0], 4)

    #Sharpe
    sharpe = acr/stdev
    # sharpe = acr/stdev
    summary[fund_price][4] = round(sharpe[0], 4)
    
    #Sortino Ratio
    tt2 = ADD
    sortino = acr/tt2
    # sortino = acr/ADD
    summary[fund_price][5] = round(sortino[0], 4)
    
    #Calmar Ratio
    calmar = acr/abs(mdd)
    summary[fund_price][6] = round(calmar[0], 4)
    
# summary.to_csv('Full_correlation.csv')
summary

,GM_5_price.csv,GM_10_price.csv,GM_20_price.csv,GM_30_price.csv,GM_50_price.csv,GM_100_price.csv,GM_200_price.csv,GM_300_price.csv,GM_500_price.csv
Annualized compound return,0.1428,0.105,0.1726,0.2024,0.2813,0.2353,0.1861,0.1699,0.138
Annualized Standard Deviation,0.2311,0.2033,0.1892,0.186,0.1838,0.1798,0.1759,0.1729,0.1653
Annualized Downside Deviation,0.1675,0.1466,0.1362,0.1341,0.1325,0.1298,0.1274,0.1255,0.1212
Maximum Drawdown,-0.6539,-0.5837,-0.5549,-0.5986,-0.5251,-0.5566,-0.5561,-0.5692,-0.5973
Sharpe Ratio,0.618,0.5164,0.9124,1.0883,1.5302,1.3083,1.0582,0.9822,0.8346
Sortino Ratio,0.8527,0.7161,1.2674,1.5098,2.123,1.8126,1.4604,1.3533,1.1387
Calmar Ratio,0.2184,0.1798,0.3111,0.3382,0.5358,0.4227,0.3346,0.2984,0.231


In [12]:
path = 'subtraction/weight/'
file_list = os.listdir(path)
weight_list = [file for file in file_list if file.endswith('weight.csv')]
weight_list.sort(key=lambda x: int(x.split('_')[1]))
weight_list

['GM_5_weight.csv',
 'GM_10_weight.csv',
 'GM_20_weight.csv',
 'GM_30_weight.csv',
 'GM_50_weight.csv',
 'GM_100_weight.csv',
 'GM_200_weight.csv',
 'GM_300_weight.csv',
 'GM_500_weight.csv']

In [14]:
for file in weight_list:
    turnover = pd.DataFrame(index=pd.to_datetime(checklist), columns=['Turnover'])
    weight_data = pd.read_csv(path+file,index_col='Unnamed: 0')
    shift =  np.log(SP500_FULL/SP500_FULL.shift(periods=1))
    for rebalancedate_idx in range(len(checklist))[:-1]:
        print('                    ')
        print('-------' + str(checklist[rebalancedate_idx])[:10] + '-------')
        tmp_idx = str(checklist[rebalancedate_idx])[:10]
        portfolio_weight = weight_data.loc[tmp_idx]
        next_tmp_idx = str(checklist[rebalancedate_idx+1])[:10]
        next_portfolio_weight = weight_data.loc[next_tmp_idx]
        
        if(rebalancedate_idx < len(checklist) - 1) : # 마지막 리밸런싱 시점이 아닌 경우
            sliced_data = shift[str(checklist[rebalancedate_idx])[:10]:str(checklist[rebalancedate_idx + 1])[:10]].copy() # 현 리밸런싱 시점에서 다음 리밸런싱 시점 사이의 종가 데이터를 가져온다. (다음 리밸런싱 시점까지 포함됨)

            if str(checklist[rebalancedate_idx + 1])[:10] in sliced_data.index: 
                sliced_data = sliced_data[:-1].dropna(axis = 1,how='all') # sliced_data에 다음 리밸런싱 시점이 포함이 되므로 해당 일 제외함.
            else:
                sliced_data = sliced_data[:].dropna(axis = 1,how='all') # sliced_data에 다음 리밸런싱 시점이 포함이 안되는 경우가 존재함 (ex: 3월 1일). 이 경우, [:-1]로 슬라이싱하게 되면 하루가 덜 투자되는 상황이 발생함. 

        else: # 마지막 리밸런싱 시점인 경우, 현 리밸런싱 시점에서 끝까지의 종가 데이터를 가져옴.
            sliced_data = shift[str(checklist[rebalancedate_idx])[:10]:str(enddate)].copy()
            sliced_data = sliced_data[:].dropna(axis = 1,how='all')
        
        sliced_data.index = pd.to_datetime(sliced_data.index)
        monthly_cumulative_return = np.exp(sliced_data.cumsum(axis=0).iloc[-1,:]) - 1
        total_value = 1+np.sum(portfolio_weight*monthly_cumulative_return)
        weight_at_the_end_of_the_month = (portfolio_weight*(1+monthly_cumulative_return))/total_value
        
        Current = weight_at_the_end_of_the_month.dropna().index
        Next = next_portfolio_weight.dropna().index
        TMP = list(set(Next.append(Current))) # 재조정시점에 사거나, 팔아야 될 모든 종목의 리스트

        tmp_weight = pd.DataFrame(index=TMP,columns=['weight'])
        
        for asset in tmp_weight.index:
            if asset not in Current:
                tmp_weight.loc[asset,'weight'] = next_portfolio_weight.dropna()[asset]
            if asset not in Next:
                tmp_weight.loc[asset,'weight'] = -weight_at_the_end_of_the_month.dropna()[asset]
            if (asset in Current) and (asset in Next) :
                tmp_weight.loc[asset,'weight'] = next_portfolio_weight.dropna()[asset] - weight_at_the_end_of_the_month.dropna()[asset]
        print(np.abs(tmp_weight).sum(axis=0)[0])
        turnover.loc[next_tmp_idx,'Turnover'] = np.abs(tmp_weight).sum(axis=0)[0]
    
    # Ensure the directory exists
    os.makedirs('subtraction/turnover', exist_ok=True)
    turnover.to_csv('subtraction/turnover/'+file.replace('_weight.csv','')+'_turnover.csv')

                    
-------2001-01-01-------
1.9999999999999996
                    
-------2001-02-01-------
1.7113885368817345
                    
-------2001-03-01-------
1.9999999999999998
                    
-------2001-04-01-------
2.0
                    
-------2001-05-01-------
1.5045399640905748
                    
-------2001-06-01-------
1.9999999999999996
                    
-------2001-07-01-------
1.9999999999999998
                    
-------2001-08-01-------
1.4720506588154083
                    
-------2001-09-01-------
1.9999999999999996
                    
-------2001-10-01-------
1.6140497151624273
                    
-------2001-11-01-------
2.0
                    
-------2001-12-01-------
1.260235099780616
                    
-------2002-01-01-------
2.0
                    
-------2002-02-01-------
1.9999999999999993
                    
-------2002-03-01-------
1.9999999999999996
                    
-------2002-04-01-------
1.9999999999999996
      

In [15]:
path = 'subtraction/turnover/'
file_list = os.listdir(path)
turnover_list = [file for file in file_list]
turnover_list.sort(key=lambda x: int(x.split('_')[1]))
turnover_list

['GM_5_turnover.csv',
 'GM_10_turnover.csv',
 'GM_20_turnover.csv',
 'GM_30_turnover.csv',
 'GM_50_turnover.csv',
 'GM_100_turnover.csv',
 'GM_200_turnover.csv',
 'GM_300_turnover.csv',
 'GM_500_turnover.csv']

In [17]:
summary = pd.DataFrame(columns = turnover_list, index = ['Average','Standard deviation'])

for tmp in turnover_list:
    data = pd.read_csv(path+tmp)['Turnover']
    summary[tmp][0] = round(data.mean(), 4)
    summary[tmp][1] = round(data.std(), 4)

summary

,GM_5_turnover.csv,GM_10_turnover.csv,GM_20_turnover.csv,GM_30_turnover.csv,GM_50_turnover.csv,GM_100_turnover.csv,GM_200_turnover.csv,GM_300_turnover.csv,GM_500_turnover.csv
Average,1.9233,1.8684,1.8032,1.7548,1.6348,1.3762,0.9763,0.7084,0.2987
Standard deviation,0.1726,0.1783,0.1602,0.155,0.1978,0.1974,0.1769,0.142,0.093


In [18]:
path = 'subtraction/price/'
file_list = os.listdir(path)
price_list = [file for file in file_list]
price_list.sort(key=lambda x: int(x.split('_')[-2]))

path2 = 'subtraction/turnover/'
file_list = os.listdir(path2)
turnover_list = [file for file in file_list]
turnover_list.sort(key=lambda x: int(x.split('_')[1]))

Adjusted Portfolio Value with Transaction Cost

In [20]:
for fund_price,turnover in zip(price_list,turnover_list):
    
    price = pd.read_csv(path+fund_price,index_col='Unnamed: 0')
    price.index = pd.to_datetime(price.index)
    monthly_turnover = pd.read_csv(path2+turnover,index_col='Unnamed: 0')
    monthly_turnover.index = pd.to_datetime(checklist)
    monthly_turnover = monthly_turnover[1:]
    
    for idx in monthly_turnover.index:
        
        rebal_time = price.index[price.index >= idx][0]
        transaction_cost = price.loc[rebal_time,:][0]*monthly_turnover.loc[idx,:][0] * 0.001
        price.loc[rebal_time:,:] = price.loc[rebal_time:,:] - transaction_cost

    # Ensure the directory exists
    os.makedirs('subtraction/price_with_transaction', exist_ok=True)
    price.to_csv('subtraction/price_with_transaction/'+fund_price.replace('_price.csv','')+'_price.csv')

In [21]:
path = 'subtraction/price_with_transaction/'
file_list = os.listdir(path)
price_list = [file for file in file_list]
price_list.sort(key=lambda x: int(x.split('_')[-2]))
price_list

['GM_5_price.csv',
 'GM_10_price.csv',
 'GM_20_price.csv',
 'GM_30_price.csv',
 'GM_50_price.csv',
 'GM_100_price.csv',
 'GM_200_price.csv',
 'GM_300_price.csv',
 'GM_500_price.csv']

Table 1. Global Motion Subtracted Correlation Portfolio Performances

In [ ]:
summary = pd.DataFrame(columns = price_list, index = ['Annualized compound return','Annualized Standard Deviation',\
                                                    'Annualized Downside Deviation','Maximum Drawdown','Sharpe Ratio','Sortino Ratio','Calmar Ratio'])
for fund_price in price_list:
    data = pd.read_csv(path+fund_price,index_col='Unnamed: 0')
    data.index = pd.to_datetime(data.index)

    log_ret = np.log(data/data.shift(periods=1))
    log_ret = log_ret.dropna()
    log_ret.index = data.index[1:]

    #Annualized coupound return
    cumsum = (data.values[-1] /data.values[0] -1)
    acr = cumsum/(len(log_ret.values) / 252)
    summary[fund_price][0] = round(acr[0], 4)

    #Annualized Standard Deviation
    stdev = np.std(log_ret.values)*np.sqrt(252)
    summary[fund_price][1] = round(stdev, 4)

    #Annualized Downside Deviation
    ADD = downside_risk(log_ret.values)
    summary[fund_price][2] = round(ADD, 4)

    #MDD
    mdd = MDD(log_ret)
    summary[fund_price][3] = round(mdd[0], 4)

    #Sharpe
    sharpe = acr/stdev
    # sharpe = acr/stdev
    summary[fund_price][4] = round(sharpe[0], 4)
    
    #Sortino Ratio
    tt2 = ADD
    sortino = acr/tt2
    # sortino = acr/ADD
    summary[fund_price][5] = round(sortino[0], 4)
    
    #Calmar Ratio
    calmar = acr/abs(mdd)
    summary[fund_price][6] = round(calmar[0], 4)
    
summary

,GM_5_price.csv,GM_10_price.csv,GM_20_price.csv,GM_30_price.csv,GM_50_price.csv,GM_100_price.csv,GM_200_price.csv,GM_300_price.csv,GM_500_price.csv
Annualized compound return,0.1056,0.0751,0.1327,0.1618,0.2364,0.2008,0.1638,0.1546,0.1317
Annualized Standard Deviation,0.2752,0.2434,0.2231,0.2194,0.2117,0.2036,0.1932,0.1859,0.1705
Annualized Downside Deviation,0.1992,0.1759,0.1614,0.1589,0.1534,0.1475,0.1404,0.1352,0.1251
Maximum Drawdown,-0.7779,-0.7173,-0.6393,-0.6865,-0.6019,-0.6239,-0.6048,-0.608,-0.6127
Sharpe Ratio,0.3838,0.3083,0.5946,0.7374,1.1163,0.9863,0.8479,0.8313,0.7723
Sortino Ratio,0.5301,0.4267,0.8219,1.0184,1.5412,1.3615,1.1667,1.143,1.0528
Calmar Ratio,0.1358,0.1046,0.2075,0.2357,0.3927,0.3218,0.2709,0.2542,0.215
